# Extract and align data from Onix, Harp, Sleap, and photometry
## Cohort 1 and 2 working, Cohort 0: onix_digital Clock column is 0, explore why and/or use timestamps instead 

In [ ]:
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
import harp
import plotly.express as px

from harp_resources import process, utils
from sleap import load_and_process as lp

In [ ]:
cohort0 = False #only read harp data when it exists, not in Cohort0 
cohort2 = False

#Cohort 1 vestibular mismatch, multiple OnixDigital files 
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2718-2024-12-12T13-28-14') #multiple onix_digital file

#Cohort 1 with clock accumulation issue marked on google sheet, seems fine though
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2719-2024-12-12T13-59-38') #multiple onix_digital file

#Cohort 1 visual mismatch 
data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/Visual_mismatch_day3/B6J2718-2024-12-10T12-57-02') 

#Cohort 0 (no OnixHarp in this Cohort)
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort0/Cohort0_GCaMP_example/B3M3xx-2024-08-08T10-05-26')
#cohort0 = True

#Cohort 2 N.B. no videodata in this test set 
#cohort2 = True
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort2_like_test_data/2025-01-13T15-47-26')

#Cohort 2 longer test YES OnixHarp! 
#N.B. no photometry in this test set (neitjer videos, but yes video_data)
#cohort2 = True
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort2_test_longer/2025-02-10T08-18-59')
 
photometry_path = data_path.parent / f"{data_path.name}_processedData" / "photometry"

h1_datafolder = data_path / 'HarpDataH1' #only if reading separate registers
# h2_datafolder = data_path / 'HarpDataH2' #only if reading separate registers

In [ ]:
#h1 and h2 only needed if timestamps are readed separately and not as all harp_streams
h1_reader = harp.create_reader('harp_resources/h1-device.yml', epoch=harp.REFERENCE_EPOCH)
# h2_reader = harp.create_reader('harp_resources/h2-device.yml', epoch=harp.REFERENCE_EPOCH)

session_settings_reader = utils.SessionData("SessionSettings")
experiment_events_reader = utils.TimestampedCsvReader("ExperimentEvents", columns=["Event"])
onix_framecount_reader = utils.TimestampedCsvReader("OnixAnalogFrameCount", columns=["Index"])
#photometry_reader = utils.PhotometryReader("Processed_fluorescence")
video_reader1 = utils.Video("VideoData1")
video_reader2 = utils.Video("VideoData2")
onix_digital_reader = utils.OnixDigitalReader("OnixDigital", columns=["Value.Clock", "Value.HubClock", 
                                                                         "Value.DigitalInputs",
                                                                         "Seconds"])
onix_harp_reader = utils.TimestampedCsvReader("OnixHarp", columns=["Clock", "HubClock", "HarpTime"])

In [ ]:
%%time
#read metadata in 2 different ways (to df or to dict, to decide which one is better in the future)
print ("Loading session settings")
session_settings = utils.load_2(session_settings_reader, data_path) #Andrew's, creates ugly df, but used in further analysis code
#session_settings = utils.read_SessionSettings(data_path) #Hilde's, creates prety dict, not aware of multiple files

# read experiment events, video, processed photometry 
print ("Loading experiment events")
experiment_events = utils.load_2(experiment_events_reader, data_path)

print ("Loading processed fluorescence")
photometry_data=pd.read_csv(str(photometry_path)+'/Processed_fluorescence.csv')
print ("Loading processed fluorescence info")
photometry_info=pd.read_csv(str(photometry_path)+'/Info.csv')
print ("Loading processed fluorescence events")
photometry_events=pd.read_csv(str(photometry_path)+'/Events.csv')

if not cohort2:
    print ("Loading video data 1")
    video_data1 = utils.load_2(video_reader1, data_path)
    print ("Loading video data 2")
    video_data2 = utils.load_2(video_reader2, data_path)

# read Onix data 
print ("Loading OnixDigital")
onix_digital = utils.load_2(onix_digital_reader, data_path)
print ("Loading OnixAnalogFrameClock")
onix_analog_framecount = utils.load_2(onix_framecount_reader, data_path)
print ("Loading OnixAnalogClock")
onix_analog_clock = utils.read_OnixAnalogClock(data_path)
print ("Loading OnixAnalogData")
onix_analog_data = utils.read_OnixAnalogData(data_path, channels = [0], binarise=True) #channels is a list of AI lines, 0-11

#read harp streams and separate registers if needed 
print ("Loading H1 and H2 streams as dict or df")
harp_streams = utils.load_registers(data_path, dataframe = True) #loads as df, or if False, as dict 

#read syncronising signal between HARP and ONIX
if not cohort0:
    print ("Loading OnixHarp")
    onix_harp = utils.load_2(onix_harp_reader, data_path)
    # removing possible outliers 
    onix_harp = utils.detect_and_remove_outliers(
    df=onix_harp,
    x_column="HarpTime",
    y_column="Clock",
    verbose=False  # True prints all outliers
    )
    onix_harp["HarpTime"] = onix_harp["HarpTime"] + 1 # known issue with current version of ONIX, harp timestamps lag 1 second
    print ("Warning: HarpTime +1s to account for know issue with ONIX")

# print (" ")
# print ("loading separate registers from H1 and H2 data")
print ("Loading camera triggers")
camera_triggers = utils.load_harp(h1_reader.Cam0Event, h1_datafolder) #assumes Cam0 triggers both cameras
print ("Loading flow sensor data")
flow_sensor = utils.load_harp(h1_reader.OpticalTrackingRead, h1_datafolder)
print ("Done Loading")

### DEV align Onix, HARP and Photometry data 

In [ ]:
import importlib
importlib.reload(utils)
importlib.reload(process) # Forces Python to reload the updated module
None

In [ ]:
%%time
(
    conversions, 
    photometry_sync_events, 
    harp_to_onix_clock, 
    onix_time_to_photometry, 
    onix_to_harp_timestamp,
    photometry_to_harp_time
) = process.photometry_harp_onix_synchronisation(
    onix_analog_data=onix_analog_data,
    onix_analog_clock=onix_analog_clock,
    onix_analog_framecount=onix_analog_framecount,
    onix_digital=onix_digital,
    onix_harp=onix_harp,
    photometry_events=photometry_events,
    verbose=True
)


In [ ]:
photometry_data


In [ ]:
idx = 0
sec_start = block_halts.index[idx]
sec_stop = block_halts.index[idx+1]
min_time = sec_start - pd.DateOffset(seconds=1)
max_time = sec_stop + pd.DateOffset(seconds=0.5)

plt.figure()

# Interpolate or drop NaNs for Optical Tracking Data
optical_x = harp_streams['OpticalTrackingRead0X(46)'].loc[min_time:max_time].dropna()
optical_y = harp_streams['OpticalTrackingRead0Y(46)'].loc[min_time:max_time].dropna()

# Plot Optical Tracking Readout (Flow) with NaN-handling
plt.plot(optical_x, label='Flow X')
plt.plot(optical_y, label='Flow Y')

# Extract camera triggers and drop NaNs
camera_triggers = harp_streams['Cam0Event(32)'].dropna()

# Plot Camera Triggers as scatter points
plt.scatter(
    camera_triggers[min_time:max_time].index,
    np.ones((1, len(camera_triggers[min_time:max_time]))) * -10, 
    s=1, c='k', label='Camera Trigger'
)

# Mark the halt command region
plt.axvspan(sec_start, sec_start + pd.DateOffset(seconds=0.1), color='black', alpha=0.2, label='Block Start')

# Formatting
plt.xlabel("Time")
plt.ylabel("Tracking Readout")
plt.legend()
plt.title("Optical Flow and Camera Triggers")
plt.show()

# overlay the onix photodiode signal in converted time
onix_sec_start_time = harp_to_onix_clock(block_halts.iloc[idx]["Seconds"] - 1)
onix_sec_start_index = np.where(onix_analog_clock >= onix_sec_start_time)[0][0]

onix_sec_stop_time = harp_to_onix_clock(block_halts.iloc[idx+1]["Seconds"])
onix_sec_stop_index = np.where(onix_analog_clock >= onix_sec_stop_time)[0][0]
plt.plot(
    onix_to_harp_timestamp(onix_analog_clock[onix_sec_start_index:onix_sec_stop_index]),
    onix_analog_data[onix_sec_start_index:onix_sec_stop_index],  # Remove the ", 0"
    label='Photodiode'
)
# overlay photometry
photometry_sec_start_time = onix_time_to_photometry(onix_sec_start_time)
photometry_sec_stop_time = onix_time_to_photometry(onix_sec_stop_time)

# Ensure TimeStamp is the index
if "TimeStamp" in photometry_data.columns:
    photometry_data = photometry_data.set_index("TimeStamp")

# Now filter safely
photometry_sec = photometry_data.loc[photometry_sec_start_time:photometry_sec_stop_time]

#photometry_sec = photometry_data[0].loc[(photometry_data[0].index >= photometry_sec_start_time) & (photometry_data[0].index <= photometry_sec_stop_time)]

plt.plot(photometry_to_harp_time(photometry_sec.index), photometry_sec['dfF_560'], label='dfF_560')

plt.legend()


In [ ]:
print(onix_analog_data.shape)  # Check the number of dimensions


In [ ]:
# example, see harp data, experiment events, onix data for a given window, synchronised.
# where did the LinearRegularMismatch block start (approximately)?
#block_start = experiment_events[experiment_events["Event"].eq("DrumBase block started")]
block_start = experiment_events[experiment_events["Event"].eq("DrumWithReverseHalt block started")]
print(block_start)

# Get the first 20 halt times after this block started
block_halts = experiment_events[(experiment_events["Event"].eq("Apply halt: 2s")) & (experiment_events.index > block_start.index[0])].iloc[0:20]

# Plot flow sensor and camera triggers during given halt period
idx = 0
sec_start = block_halts.index[idx]
sec_stop = block_halts.index[idx+1]
min_time = sec_start - pd.DateOffset(seconds=1)
max_time = sec_stop + pd.DateOffset(seconds=0.5)

plt.figure()
plt.plot(flow_sensor['OpticalTrackingRead0X'][min_time:max_time], label='Flow X')
plt.plot(flow_sensor['OpticalTrackingRead0Y'][min_time:max_time], label='Flow Y')
plt.scatter(camera_triggers[min_time:max_time].index, np.ones((1, len(camera_triggers[min_time:max_time]))) * -10, s=1, c='k', label='Camera Trigger')
plt.axvspan(sec_start, sec_start + pd.DateOffset(seconds=0.1), color='black', alpha=0.2, label='Halt Command')

# overlay the onix photodiode signal in converted time
onix_sec_start_time = harp_to_onix_clock(block_halts.iloc[idx]["Seconds"] - 1)
onix_sec_start_index = np.where(analog_clock >= onix_sec_start_time)[0][0]

onix_sec_stop_time = harp_to_onix_clock(block_halts.iloc[idx+1]["Seconds"])
onix_sec_stop_index = np.where(analog_clock >= onix_sec_stop_time)[0][0]
plt.plot(onix_to_harp_timestamp(analog_clock[onix_sec_start_index:onix_sec_stop_index]), analog_data[onix_sec_start_index:onix_sec_stop_index, 0], label='Photodiode')

# overlay photometry
photometry_sec_start_time = onix_time_to_photometry(onix_sec_start_time)
photometry_sec_stop_time = onix_time_to_photometry(onix_sec_stop_time)
photometry_sec = photometry[0].loc[(photometry[0].index >= photometry_sec_start_time) & (photometry[0].index <= photometry_sec_stop_time)]

plt.plot(photometry_to_harp_time(photometry_sec.index), photometry_sec['CH1-410'], label='CH1-410')

plt.legend()

In [ ]:
import sys

# Function to get memory usage of all variables in MB
def get_all_memory_usage():
    memory_usage = {var: sys.getsizeof(value) for var, value in globals().items()}
    sorted_memory_usage = sorted(memory_usage.items(), key=lambda x: x[1], reverse=True)
    
    print("Variable Memory Usage (in MB):")
    for var, size in sorted_memory_usage:
        print(f"{var}: {size / (1024**2):.2f} MB")

# Call function to display memory usage
get_all_memory_usage()
